# Configuration of the architecture of the GO hidden layers of GraphGONet (step 1)

## Summary
The following notebook will extract the annotations between the input genes and the GO terms. <br>
Note that the variable **SUBONTOLOGY** indicates the subontology considered as the structure of the GO layers. By default, it is BP to follow the paper but MF or CC can be chosen. <br>
The variable **DATASET** needs to be set based on which dataset you are using. The input features are different between the two datasets used in the article, so it will not be the same annotation database we will refer to. If you desire to apply the method on another dataset not studied in the article, you should check the type and the version of the annotation database (see variable *annot_ref_db*). <br>
You should indicate your repository where the data should be saved or loaded.

#### Load packages

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.rinterface as rinterface
import rpy2.robjects.help as rh

In [4]:
base = importr('base')
utils = importr("utils")
biomanager = importr("BiocManager")
annotate = importr("annotate")
godb = importr("GO.db")
AnnotationDbi = importr("AnnotationDbi")
biomaRt = importr("biomaRt")
grdevices = importr('grDevices')
ah = importr("AnnotationHub")
org = importr("org.Hs.eg.db")

R[write to console]: Error in (function ()  : 
  org.Hs.egPFAM is defunct. Please use select() if you need access to
  PFAM or PROSITE accessions.

R[write to console]: De plus : 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  les bibliothèques ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ ne contiennent aucun package

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  les bibliothèques ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ ne contiennent aucun package

R[write to console]: 3: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  les bibliothèques ‘/usr/local/lib/R/site-librar

In [5]:
robjects.r('''
               f <- function() {

                   library("hgu133plus2.db")
                }
            ''')
r_f = robjects.globalenv['f']
r_f()

R[write to console]: 



'hgu133pl...,'org.Hs.e...,'Annotati...,...,'datasets','methods','base'


In [6]:
packageVersion = robjects.r['package.version']

In [7]:
sessionInfo = robjects.r['sessionInfo']

In [8]:
res=sessionInfo()
print(res)

R version 4.0.3 (2020-10-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=fr_FR.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=fr_FR.UTF-8        LC_COLLATE=fr_FR.UTF-8    
 [5] LC_MONETARY=fr_FR.UTF-8    LC_MESSAGES=fr_FR.UTF-8   
 [7] LC_PAPER=fr_FR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=fr_FR.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] parallel  stats4    tools     stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] hgu133plus2.db_3.2.3 org.Hs.eg.db_3.12.0  AnnotationHub_2.22.0
 [4] BiocFileCache_1.14.0 dbplyr_2.0.0         biomaRt_2.46.0      
 [7] GO.db_3.12.1         annotate_1.68.0      XML_3.99-0.5        
[10] AnnotationDbi_1.52.0 IRan

#### Set environnement

In [9]:
SUBONTOLOGY = "BP"

In [10]:
DATASET=input("Which dataset are you using? TCGA or microarray")

Which dataset are you using? TCGA or microarray TCGA


In [11]:
dir_files = "../files" #can be modified to your own path

In [11]:
dir_data = "../data" #can be modified to your own path

In [20]:
filename = os.path.join(dir_data,"id_genes.npz") #corresponds to the column names of the dataset

Set the annotation database that will be used as a reference to identify the annotations between the gene ID and GO terms.

In [14]:
annot_ref_db = org.org_Hs_eg_db if DATASET=="TCGA" else robjects.r('hgu133plus2.db')

In [15]:
keyRef = "ENSEMBL" if DATASET=="TCGA" else "PROBE"

In [26]:
def getInformations(annot_ref,inputId,type_var):
    return AnnotationDbi.select(x=annot_ref, keys=inputId, columns=robjects.StrVector(["ENTREZID","GO"]),keytype=robjects.StrVector([type_var]))

### 1. Collect the identifier of the variables in the dataset

In [34]:
loaded = np.load(filename,allow_pickle=True)
list_genes = loaded["genes"].astype(str).tolist()
list_genes

['ENSG00000000003',
 'ENSG00000000005',
 'ENSG00000000419',
 'ENSG00000000457',
 'ENSG00000000460',
 'ENSG00000000938',
 'ENSG00000000971',
 'ENSG00000001036',
 'ENSG00000001084',
 'ENSG00000001167',
 'ENSG00000001460',
 'ENSG00000001461',
 'ENSG00000001497',
 'ENSG00000001561',
 'ENSG00000001617',
 'ENSG00000001626',
 'ENSG00000001629',
 'ENSG00000001630',
 'ENSG00000001631',
 'ENSG00000002016',
 'ENSG00000002079',
 'ENSG00000002330',
 'ENSG00000002549',
 'ENSG00000002586',
 'ENSG00000002587',
 'ENSG00000002726',
 'ENSG00000002745',
 'ENSG00000002746',
 'ENSG00000002822',
 'ENSG00000002834',
 'ENSG00000002919',
 'ENSG00000002933',
 'ENSG00000003056',
 'ENSG00000003096',
 'ENSG00000003137',
 'ENSG00000003147',
 'ENSG00000003249',
 'ENSG00000003393',
 'ENSG00000003400',
 'ENSG00000003402',
 'ENSG00000003436',
 'ENSG00000003509',
 'ENSG00000003756',
 'ENSG00000003987',
 'ENSG00000003989',
 'ENSG00000004059',
 'ENSG00000004139',
 'ENSG00000004142',
 'ENSG00000004399',
 'ENSG00000004455',


In [35]:
NB_VARS=len(list_genes) 
NB_VARS

56602

### 2. Mapping with the Gene Ontology

In [37]:
df_All = getInformations(annot_ref=annot_ref_db,inputId=robjects.StrVector(list_genes),type_var=keyRef) # 

R[write to console]: 'select()' returned 1:many mapping between keys and columns



In [38]:
df_All.to_csvfile(os.path.join(dir_files,"annotations-gene-GO.csv"))

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fec80543b48> [RTYPES.NILSXP]

In [39]:
%%time
tmp = pd.read_csv(os.path.join(dir_files,"annotations-gene-GO.csv"),dtype={"GO":object,"ENTREZID":object})

CPU times: user 178 ms, sys: 7.24 ms, total: 185 ms
Wall time: 183 ms


In [40]:
tmp.head()

,ENSEMBL,ENTREZID,GO,EVIDENCE,ONTOLOGY
1,ENSG00000000003,7105,GO:0005515,IPI,MF
2,ENSG00000000003,7105,GO:0005887,IBA,CC
3,ENSG00000000003,7105,GO:0039532,IMP,BP
4,ENSG00000000003,7105,GO:0043123,HMP,BP
5,ENSG00000000003,7105,GO:0070062,HDA,CC


In [41]:
tmp.shape

(372639, 5)

In [42]:
tmp = tmp.dropna()

### 3. Select a subontology

In [43]:
onto_bp = tmp[tmp.ONTOLOGY == SUBONTOLOGY]

In [44]:
onto_bp.shape

(158050, 5)

In [51]:
NB_VARS_REAL_LINKED_TO_GO = len(onto_bp[keyRef].unique())
NB_VARS_REAL_LINKED_TO_GO

18427

In [52]:
("Number of genes missing :", NB_VARS - NB_VARS_REAL_LINKED_TO_GO)

('Number of genes missing :', 38175)

In [53]:
(NB_VARS-NB_VARS_REAL_LINKED_TO_GO)/NB_VARS*100

67.44461326454896